# Querying from Mongo DataBase

In this notebook, we'll make querys to our Mongo database and create a pandas dataframe. 

-----

**First things first, let's import some things...**

In [1]:
import json
import numpy as np
import pandas as pd
from pymongo import MongoClient
from unicodedata import normalize

<br>

**Creating a Mongo Client**

In [2]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client['pbmnet']
curriculos = db.curriculos

In [65]:
def Unest_Titulo_Artigo(cell):
    return cell['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']['DADOS-BASICOS-DO-ARTIGO']['@TITULO-DO-ARTIGO'].strip()

def Unest_Endereco_Nome(cell):
    try:
        value = cell['ENDERECO']['ENDERECO-PROFISSIONAL']['@NOME-INSTITUICAO-EMPRESA']
    except:
        value = None
    return value
def Unest_Endereco_UF_Insituicao(cell):
    try:
        value = cell['ENDERECO']['ENDERECO-PROFISSIONAL']['@UF']
    except:
        value = None
    return value
def Unest_Endereco_Pais_Insituicao(cell):
    try:
        value = cell['ENDERECO']['ENDERECO-PROFISSIONAL']['@PAIS']
    except:
        value = None
    return value

def Unest_Endereco_Codigo(cell):
    try:
        value = cell['ENDERECO']['ENDERECO-PROFISSIONAL']['@CODIGO-INSTITUICAO-EMPRESA']
    except:
        value = None
    return value

def Unest_Nacionalidade(cell):
    try:
        value = cell['@PAIS-DE-NACIONALIDADE']
    except:
        value = None
    return value

def Unest_Ano_Artigo(cell):
    return cell['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']['DADOS-BASICOS-DO-ARTIGO']['@ANO-DO-ARTIGO'].strip()

def Unest_Palavras_Chave(cell):
    try:
        value = cell['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']['PALAVRAS-CHAVE']
    except:
        value = None
    return value

def Unest_Journal(cell):
    try:
        value = cell['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']['DETALHAMENTO-DO-ARTIGO']['@TITULO-DO-PERIODICO-OU-REVISTA'].strip()
    except:
        value = None
    return value

def Unest_Area_Atuacao(cell):
    try:
        value = cell['AREAS-DE-ATUACAO']['AREA-DE-ATUACAO'][0]
    except:
        try:
            value = cell['AREAS-DE-ATUACAO']['AREA-DE-ATUACAO']
        except:
            value = None
            return value
    return value

In [26]:
def Separar_Palavras_Chave(cell):
    if cell == None:
        return None
    return '|'.join([i for i in cell.values()])

In [78]:
def Unest_GrandeArea(cell):
    try:
        value = cell[0]['@NOME-GRANDE-AREA-DO-CONHECIMENTO']
    except:
        try:
            value = cell['@NOME-GRANDE-AREA-DO-CONHECIMENTO']
        except:
            return 'CIENCIAS_EXATAS_E_DA_TERRA'
    return value

def Unest_Area(cell):
    try:
        value = cell[0]['@NOME-DA-AREA-DO-CONHECIMENTO']
    except:
        try:
            value = cell['@NOME-DA-AREA-DO-CONHECIMENTO']
        except:
            return 'Matemática'
    return value

def Unest_SubArea(cell):
    try:
        value = cell[0]['@NOME-DA-SUB-AREA-DO-CONHECIMENTO']
    except:
        try:
            value = cell['@NOME-DA-SUB-AREA-DO-CONHECIMENTO']
        except:
            return None
    return value

def Unest_Especialidade(cell):
    try:
        value = cell[0]['@NOME-DA-ESPECIALIDADE']
    except:
        try:
            value = cell['@NOME-DA-ESPECIALIDADE']
        except:
            return None
    return value

<br>

## Collecting Authors

In [27]:
unwind2 = list(curriculos.aggregate([
                           {'$project':{
        '_id':1,
        '@DATA-ATUALIZACAO':1,
        '@NUMERO-IDENTIFICADOR':1,
        'DADOS-GERAIS.@NOME-COMPLETO': 1,
        'DADOS-GERAIS.AREAS-DE-ATUACAO.AREA-DE-ATUACAO': 1,
        'DADOS-GERAIS.ENDERECO.ENDERECO-PROFISSIONAL.@NOME-INSTITUICAO-EMPRESA':1,
        'DADOS-GERAIS.ENDERECO.ENDERECO-PROFISSIONAL.@CODIGO-INSTITUICAO-EMPRESA':1,
        'DADOS-GERAIS.ENDERECO.ENDERECO-PROFISSIONAL.@PAIS':1,
        'DADOS-GERAIS.ENDERECO.ENDERECO-PROFISSIONAL.@UF':1,
        'DADOS-GERAIS.@PAIS-DE-NACIONALIDADE': 1,
        'DADOS-GERAIS.@NOME-EM-CITACOES-BIBLIOGRAFICAS': 1
                            }}      ]))

In [33]:
unwind2[10]['DADOS-GERAIS']['AREAS-DE-ATUACAO']['AREA-DE-ATUACAO']

{'@SEQUENCIA-AREA-DE-ATUACAO': '1',
 '@NOME-GRANDE-AREA-DO-CONHECIMENTO': 'CIENCIAS_EXATAS_E_DA_TERRA',
 '@NOME-DA-AREA-DO-CONHECIMENTO': 'Matemática',
 '@NOME-DA-SUB-AREA-DO-CONHECIMENTO': 'Geometria e Topologia',
 '@NOME-DA-ESPECIALIDADE': 'Geometria Diferencial'}

In [86]:
df2 = pd.DataFrame(unwind2)

df2['NOME-COMPLETO']     = df2["DADOS-GERAIS"].apply(lambda cell: cell['@NOME-COMPLETO'].strip())
df2['AREA-ATUACAO']      = df2["DADOS-GERAIS"].apply(Unest_Area_Atuacao)
df2['GRANDE-AREA']       = df2["AREA-ATUACAO"].apply(Unest_GrandeArea)
df2['AREA']              = df2["AREA-ATUACAO"].apply(Unest_Area)
df2['SUB-AREA']          = df2["AREA-ATUACAO"].apply(Unest_SubArea)
df2['ESPECIALIDADE']     = df2["AREA-ATUACAO"].apply(Unest_Especialidade)
df2['NOME-INSTITUICAO']  = df2["DADOS-GERAIS"].apply(Unest_Endereco_Nome)
df2['UF-INSTITUICAO']    = df2["DADOS-GERAIS"].apply(Unest_Endereco_UF_Insituicao)
df2['PAIS-INSTITUICAO']  = df2["DADOS-GERAIS"].apply(Unest_Endereco_Pais_Insituicao)
df2['PAIS']              = df2["DADOS-GERAIS"].apply(Unest_Nacionalidade)
df2['NOME-CITACOES']     = df2["DADOS-GERAIS"].apply(lambda cell: cell['@NOME-EM-CITACOES-BIBLIOGRAFICAS'].strip())
df2['CODIGO-INSTITUICAO']= df2["DADOS-GERAIS"].apply(Unest_Endereco_Codigo)

df2.drop(labels='DADOS-GERAIS',axis=1,inplace=True)

In [87]:
def Fix_Date_String(cell):
    return cell[0:2]+'-'+cell[2:4]+'-'+cell[4:]
df2['@DATA-ATUALIZACAO'] = pd.to_datetime(df2['@DATA-ATUALIZACAO'].apply(Fix_Date_String))


There are some duplicate authors. The code below eliminates them, leaving the record with most recent update time.

In [88]:
df2 = df2.loc[df2.groupby('NOME-COMPLETO')['@DATA-ATUALIZACAO'].idxmax(),:]

## Código abaixo checa os duplicados
# duplicates = df2.groupby(by='NOME-COMPLETO').count().reset_index()
# duplicates = list(duplicates[duplicates['_id']>1]['NOME-COMPLETO'])
# df2[df2['NOME-COMPLETO'].isin(duplicates)].sort_values(by='@DATA-ATUALIZACAO')

In [93]:
df2.head(2)

,@DATA-ATUALIZACAO,@NUMERO-IDENTIFICADOR,_id,NOME-COMPLETO,AREA-ATUACAO,GRANDE-AREA,AREA,SUB-AREA,ESPECIALIDADE,NOME-INSTITUICAO,UF-INSTITUICAO,PAIS-INSTITUICAO,PAIS,NOME-CITACOES,CODIGO-INSTITUICAO
0,2014-10-07,2558569782799336,5d31b6f7a4ef2aef4f143ec2,Aarão Lyra,"{'@SEQUENCIA-AREA-DE-ATUACAO': '1', '@NOME-GRA...",CIENCIAS_EXATAS_E_DA_TERRA,Ciência da Computação,Matemática da Computação,Matemática Intervalar,Universidade Potiguar,RN,Brasil,Brasil,"LYRA, A.;Lyra, A.;Lyra, Aarão;LYRA, A",000100000991
1,2017-08-01,3172487852109469,5d31bcc1a4ef2aef4f144f75,Abdelhakim Senhaji Hafid,"{'@SEQUENCIA-AREA-DE-ATUACAO': '1', '@NOME-GRA...",CIENCIAS_EXATAS_E_DA_TERRA,Ciência da Computação,,,,,,Marrocos,"HAFID, A. S.;A. HAFID;HAFID, ABDELHAKIM;HAFID,...",


In [90]:
df2.reset_index(drop=True,inplace=True)

In [94]:
df2.to_csv('views/Authors.csv', sep=';', encoding='latin-1')

<br>

## Collecting Papers

In [16]:
unwind3 = list(curriculos.aggregate([
                           {'$project':{
        '_id':1,
        '@NUMERO-IDENTIFICADOR':1,
        'DADOS-GERAIS.@NOME-COMPLETO': 1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DADOS-BASICOS-DO-ARTIGO.@TITULO-DO-ARTIGO': 1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DADOS-BASICOS-DO-ARTIGO.@ANO-DO-ARTIGO':1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DETALHAMENTO-DO-ARTIGO.@TITULO-DO-PERIODICO-OU-REVISTA':1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.PALAVRAS-CHAVE':1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.AUTORES': 1
                                       }},
                            {'$unwind': '$PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO'},
                          ]))

In [17]:
unwind3[0]

{'_id': ObjectId('5d31b5b9a4ef2aef4f14294d'),
 '@NUMERO-IDENTIFICADOR': '6115398975075240',
 'DADOS-GERAIS': {'@NOME-COMPLETO': 'Manuel Francisco Zuloeta Jimenez'},
 'PRODUCAO-BIBLIOGRAFICA': {'ARTIGOS-PUBLICADOS': {'ARTIGO-PUBLICADO': {'DADOS-BASICOS-DO-ARTIGO': {'@TITULO-DO-ARTIGO': 'Weak almost periodic motions, minimality and stability in impulsive semidynamical systems',
     '@ANO-DO-ARTIGO': '2014'},
    'DETALHAMENTO-DO-ARTIGO': {'@TITULO-DO-PERIODICO-OU-REVISTA': 'Journal of Differential Equations (Print)'},
    'AUTORES': [{'@NOME-COMPLETO-DO-AUTOR': 'BONOTTO, E.M.',
      '@NOME-PARA-CITACAO': 'BONOTTO, E.M.',
      '@ORDEM-DE-AUTORIA': '1',
      '@NRO-ID-CNPQ': ''},
     {'@NOME-COMPLETO-DO-AUTOR': 'Manuel Francisco Zuloeta Jimenez',
      '@NOME-PARA-CITACAO': 'JIMENEZ, M.Z.',
      '@ORDEM-DE-AUTORIA': '2',
      '@NRO-ID-CNPQ': ''}],
    'PALAVRAS-CHAVE': {'@PALAVRA-CHAVE-1': 'Sistemas semidinâmicos Impulsivos',
     '@PALAVRA-CHAVE-2': 'Weak almost periodic motions',
 

In [18]:
df3 = pd.DataFrame(unwind3)

In [19]:
def Unest_Coautores(cell):
    return cell['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']['AUTORES']

In [20]:
df3['NOME-COMPLETO']     = df3["DADOS-GERAIS"].apply(lambda cell: cell['@NOME-COMPLETO'].strip())
df3['TITULO-DO-ARTIGO']  = df3["PRODUCAO-BIBLIOGRAFICA"].apply(Unest_Titulo_Artigo)
df3['ANO-DO-ARTIGO']     = df3["PRODUCAO-BIBLIOGRAFICA"].apply(Unest_Ano_Artigo)
df3['JOURNAL']           = df3['PRODUCAO-BIBLIOGRAFICA'].apply(Unest_Journal)
df3['PALAVRAS-CHAVE-ARTIGO'] = df3["PRODUCAO-BIBLIOGRAFICA"].apply(Unest_Palavras_Chave)
df3['PALAVRAS-CHAVE-ARTIGO'] = df3['PALAVRAS-CHAVE-ARTIGO'].apply(Separar_Palavras_Chave)
df3['COAUTORES'] = df3['PRODUCAO-BIBLIOGRAFICA'].apply(Unest_Coautores)


df3.drop(labels=['PRODUCAO-BIBLIOGRAFICA','DADOS-GERAIS'],axis=1,inplace=True)

In [50]:
df3[df3['TITULO-DO-ARTIGO']==''] #Data incorrect! There is no paper

,@NUMERO-IDENTIFICADOR,_id,NOME-COMPLETO,TITULO-DO-ARTIGO,ANO-DO-ARTIGO,JOURNAL,PALAVRAS-CHAVE-ARTIGO,COAUTORES


In [48]:
df3.drop(labels=58360,inplace=True)

In [51]:
df3.shape

(138475, 8)

In [62]:
def Remove_Accents(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def Clean_Article_Name(cell):
    return Remove_Accents(cell.strip().upper().replace(' ','').replace(',','').replace('.','').replace("\"",'')).replace(")",'').replace("(",'').replace('-','').replace('!','').replace(':','').replace(';','')

df3['CHAVE_ARTIGO'] = df3['TITULO-DO-ARTIGO'].apply(Clean_Article_Name)

In [63]:
df3.to_csv('views/Papers.csv', sep=';', encoding='latin-1')

<br>

## Collecting 'Research Projects Description'

In [22]:
unwind4 = list(curriculos.aggregate([
                           {'$project':{
        '_id':1,
        '@NUMERO-IDENTIFICADOR':1,
        'DADOS-GERAIS.@NOME-COMPLETO': 1,
        'DADOS-GERAIS.ATUACOES-PROFISSIONAIS.ATUACAO-PROFISSIONAL.ATIVIDADES-DE-PARTICIPACAO-EM-PROJETO.PARTICIPACAO-EM-PROJETO.PROJETO-DE-PESQUISA.@NOME-DO-PROJETO': 1,
        'DADOS-GERAIS.ATUACOES-PROFISSIONAIS.ATUACAO-PROFISSIONAL.ATIVIDADES-DE-PARTICIPACAO-EM-PROJETO.PARTICIPACAO-EM-PROJETO.PROJETO-DE-PESQUISA.@SITUACAO': 1,
        'DADOS-GERAIS.ATUACOES-PROFISSIONAIS.ATUACAO-PROFISSIONAL.ATIVIDADES-DE-PARTICIPACAO-EM-PROJETO.PARTICIPACAO-EM-PROJETO.PROJETO-DE-PESQUISA.@NATUREZA': 1,
        'DADOS-GERAIS.ATUACOES-PROFISSIONAIS.ATUACAO-PROFISSIONAL.ATIVIDADES-DE-PARTICIPACAO-EM-PROJETO.PARTICIPACAO-EM-PROJETO.PROJETO-DE-PESQUISA.@DESCRICAO-DO-PROJETO': 1,
        'DADOS-GERAIS.ATUACOES-PROFISSIONAIS.ATUACAO-PROFISSIONAL.ATIVIDADES-DE-PARTICIPACAO-EM-PROJETO.PARTICIPACAO-EM-PROJETO.PROJETO-DE-PESQUISA.@ANO-INICIO': 1,
        'DADOS-GERAIS.ATUACOES-PROFISSIONAIS.ATUACAO-PROFISSIONAL.ATIVIDADES-DE-PARTICIPACAO-EM-PROJETO.PARTICIPACAO-EM-PROJETO.PROJETO-DE-PESQUISA.@ANO-FIM': 1
                                       }},
                            {'$unwind': '$DADOS-GERAIS.ATUACOES-PROFISSIONAIS.ATUACAO-PROFISSIONAL.ATIVIDADES-DE-PARTICIPACAO-EM-PROJETO.PARTICIPACAO-EM-PROJETO'},
                          ]))

In [23]:
def Unest_Proj_Nome(cell):
    try:
        return cell['ATUACOES-PROFISSIONAIS']['ATUACAO-PROFISSIONAL']['ATIVIDADES-DE-PARTICIPACAO-EM-PROJETO']['PARTICIPACAO-EM-PROJETO']['PROJETO-DE-PESQUISA']["@NOME-DO-PROJETO"]
    except:
        return None
    
def Unest_Proj_Ano_I(cell):
    try:
        return cell['ATUACOES-PROFISSIONAIS']['ATUACAO-PROFISSIONAL']['ATIVIDADES-DE-PARTICIPACAO-EM-PROJETO']['PARTICIPACAO-EM-PROJETO']['PROJETO-DE-PESQUISA']["@ANO-INICIO"]
    except:
        return None
def Unest_Proj_Ano_F(cell):
    try:
        return cell['ATUACOES-PROFISSIONAIS']['ATUACAO-PROFISSIONAL']['ATIVIDADES-DE-PARTICIPACAO-EM-PROJETO']['PARTICIPACAO-EM-PROJETO']['PROJETO-DE-PESQUISA']["@ANO-FIM"]
    except:
        return None
    
def Unest_Proj_Situacao(cell):
    try:
        return cell['ATUACOES-PROFISSIONAIS']['ATUACAO-PROFISSIONAL']['ATIVIDADES-DE-PARTICIPACAO-EM-PROJETO']['PARTICIPACAO-EM-PROJETO']['PROJETO-DE-PESQUISA']["@SITUACAO"]
    except:
        return None
    
def Unest_Proj_Natureza(cell):
    try:
        return cell['ATUACOES-PROFISSIONAIS']['ATUACAO-PROFISSIONAL']['ATIVIDADES-DE-PARTICIPACAO-EM-PROJETO']['PARTICIPACAO-EM-PROJETO']['PROJETO-DE-PESQUISA']["@NATUREZA"]
    except:
        return None
    
def Unest_Proj_Desc(cell):
    try:
        return cell['ATUACOES-PROFISSIONAIS']['ATUACAO-PROFISSIONAL']['ATIVIDADES-DE-PARTICIPACAO-EM-PROJETO']['PARTICIPACAO-EM-PROJETO']['PROJETO-DE-PESQUISA']["@DESCRICAO-DO-PROJETO"]
    except:
        return None

In [24]:
df4 = pd.DataFrame(unwind4)

In [25]:
df4['NOME-COMPLETO']    = df4["DADOS-GERAIS"].apply(lambda cell: cell['@NOME-COMPLETO'].strip())
df4['NOME-PROJETO']     = df4["DADOS-GERAIS"].apply(Unest_Proj_Nome)
df4['DESC-PROJETO']     = df4["DADOS-GERAIS"].apply(Unest_Proj_Desc)
df4['NATUREZA-PROJETO'] = df4["DADOS-GERAIS"].apply(Unest_Proj_Natureza)
df4['ANO-INICIO-PROJETO']     = df4["DADOS-GERAIS"].apply(Unest_Proj_Ano_I)
df4['ANO-FIM-PROJETO']     = df4["DADOS-GERAIS"].apply(Unest_Proj_Ano_F)
df4['SITUACAO-PROJETO']     = df4["DADOS-GERAIS"].apply(Unest_Proj_Situacao)

df4.drop(labels=['DADOS-GERAIS'],axis=1,inplace=True)

In [26]:
df4.to_csv('views/Research_projects.csv', sep=';', encoding='latin-1')

# Dataframes produced.